In [50]:
import os
from scipy.io import wavfile
import numpy as np
nTimeStep = 100000

MajorData = []
for subdir, dirs, files in os.walk(r'./Dataset/Audio_Files/Major/'):
    for file in files:
        filepath = subdir + os.sep + file
        samplerate, data = wavfile.read(filepath)
        if len(data) > nTimeStep:
            MajorData.append(data[0:nTimeStep])
        elif len(data) < nTimeStep:
            A = np.zeros(nTimeStep)
            A[0:len(data)] = data
            MajorData.append(A)
        else:
            MajorData.append(data)
MinorData = []
for subdir, dirs, files in os.walk(r'./Dataset/Audio_Files/Minor/'):
    for file in files:
        filepath = subdir + os.sep + file
        samplerate, data = wavfile.read(filepath)
        if len(data) > nTimeStep:
            MinorData.append(data[0:nTimeStep])
        elif len(data) < nTimeStep:
            A = np.zeros(nTimeStep)
            A[0:len(data)] = data
            MinorData.append(A)
        else:
            MinorData.append(data)            
    

In [101]:
import torch
MajorData = np.array(MajorData, dtype=np.float32)
MinorData = np.array(MinorData, dtype=np.float32)
# Split train and test
def shuffle_and_split_data(X, y, seed, test_percentage, shuffle_bool=False):
  torch.manual_seed(seed)
  # Number of samples
  N = X.shape[0]
  # Shuffle data if required
  if shuffle_bool:
    indices =torch.randperm(N)   # Get indices to shuffle data, could use torch.randperm
    print(indices)
  else:
    indices=torch.arange(N)
    #print(indices)
  X = X[indices]
  y = y[indices]
  # Split data into train/test
  test_size =int(test_percentage*N)    # Assign test datset size using 20% of samples
  X_test = X[:test_size]
  y_test = y[:test_size]
  X_train = X[test_size:]
  y_train = y[test_size:]
  return X_test, y_test, X_train, y_train
MajorY = np.zeros((np.shape(MajorData)[0],2))
MajorY[:,0] = 1
MinorY = np.zeros((np.shape(MinorData)[0],2))
MinorY[:,1] = 1
testMajor, testMajorY, trainMajor, trainMajorY = shuffle_and_split_data(MajorData, MajorY, 1, 0.2)
testMinor, testMinorY, trainMinor, trainMinorY = shuffle_and_split_data(MinorData, MinorY, 1, 0.2)

In [220]:
trainChord = np.concatenate((trainMajor, trainMinor), axis = 0)
testChord = np.concatenate((testMajor, testMinor), axis = 0)
trainChordY = np.concatenate((trainMajorY, trainMinorY), axis = 0)
testChordY = np.concatenate((testMajorY, testMinorY), axis = 0)
ds = 10
trainChordDownSample10 = trainChord[:,0::ds]
testChordDownSample10 = testChord[:,0::ds]
ds = 100
trainChordDownSample40 = trainChord[:,0::ds]
testChordDownSample40 = testChord[:,0::ds]


trainFFT = np.abs(np.fft.fft(trainChordDownSample10,1024))
testFFT = np.abs(np.fft.fft(testChordDownSample10,1024))


In [222]:
import torch.nn as nn
from torch.autograd import Variable

class VanillaRNN(nn.Module):
  def __init__(self, output_size, hidden_size, input_size):
    self.numL = 1
    super(VanillaRNN, self).__init__()
    self.hidden_size = hidden_size
    self.rnn = nn.RNN(input_size, hidden_size, num_layers = self.numL)
    self.fc = nn.Linear(self.numL*hidden_size, output_size)

  def forward(self, input):
    input = input.permute(1, 0, 2)
    h_0 =  Variable(torch.zeros(self.numL, input.size()[1], self.hidden_size))
    output, h_n = self.rnn(input, h_0)
    h_n = h_n.permute(1, 0, 2)
    h_n = h_n.contiguous().view(h_n.size()[0], h_n.size()[1]*h_n.size()[2])
    logits = self.fc(h_n)
    return logits

## Uncomment to test
#model = VanillaRNN(2, 5, trainFFT.shape[1])
model = VanillaRNN(2, 5, trainChordDownSample40.shape[1])
loss_func = nn.CrossEntropyLoss()
num_epochs = 2000
learning_rate = 0.0001
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate, weight_decay = 0.0001)
traindataset = torch.utils.data.TensorDataset(torch.Tensor(np.expand_dims(trainChordDownSample40,1)), torch.Tensor(trainChordY))
testdataset = torch.utils.data.TensorDataset(torch.Tensor(np.expand_dims(testChordDownSample40,1)), torch.Tensor(testChordY))

#traindataset = torch.utils.data.TensorDataset(torch.Tensor(np.expand_dims(trainFFT,1)), torch.Tensor(trainChordY))
#testdataset = torch.utils.data.TensorDataset(torch.Tensor(np.expand_dims(testFFT,1)), torch.Tensor(testChordY))

train_loader = torch.utils.data.DataLoader(traindataset, batch_size = 100, shuffle = True)
test_loader = torch.utils.data.DataLoader(testdataset, batch_size= 100, shuffle=False)
count = 0
loss_list = []
iteration_list = []
accuracy_list = []

for epoch in range(num_epochs):
  for datapoints, labels in train_loader:
    count += 1
    model.train()
    outputs = model(datapoints)
    loss = loss_func(outputs, labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if (count < 10) or not(count % 300):
        with torch.no_grad():
            total = 0
            correct = 0
            for datapoints, labels in test_loader:
                model.eval()
                outputs = model(datapoints)
                predictions = torch.max(outputs, 1)[1]
                truth = torch.max(labels, 1)[1]
                correct += (predictions == truth).sum().numpy()
                total += len(labels)
            accuracy = correct * 100 / total
            loss_list.append(loss.data)
            iteration_list.append(count)
            accuracy_list.append(accuracy)
            print(f'Epoch: {epoch+1:02d}, Iteration: {count:5d}, Loss: {loss.data:.4f}, Accuracy: {accuracy:.3f}%')

Epoch: 01, Iteration:     1, Loss: 0.7724, Accuracy: 53.216%
Epoch: 01, Iteration:     2, Loss: 0.7384, Accuracy: 53.216%
Epoch: 01, Iteration:     3, Loss: 0.7857, Accuracy: 53.216%
Epoch: 01, Iteration:     4, Loss: 0.7032, Accuracy: 53.216%
Epoch: 01, Iteration:     5, Loss: 0.7320, Accuracy: 53.216%
Epoch: 01, Iteration:     6, Loss: 0.7778, Accuracy: 53.216%
Epoch: 01, Iteration:     7, Loss: 0.7662, Accuracy: 53.216%
Epoch: 02, Iteration:     8, Loss: 0.7617, Accuracy: 53.216%
Epoch: 02, Iteration:     9, Loss: 0.6815, Accuracy: 53.216%
Epoch: 43, Iteration:   300, Loss: 0.7544, Accuracy: 52.632%
Epoch: 86, Iteration:   600, Loss: 0.6738, Accuracy: 53.216%
Epoch: 129, Iteration:   900, Loss: 0.6665, Accuracy: 49.708%
Epoch: 172, Iteration:  1200, Loss: 0.7052, Accuracy: 49.708%
Epoch: 215, Iteration:  1500, Loss: 0.6944, Accuracy: 52.047%
Epoch: 258, Iteration:  1800, Loss: 0.6958, Accuracy: 50.292%
Epoch: 300, Iteration:  2100, Loss: 0.6701, Accuracy: 50.292%
Epoch: 343, Iterati

In [219]:
import torch.nn.functional as F
class Net1(nn.Module):
  def __init__(self):
    super(Net1, self).__init__()
    self.hidden = nn.Linear(trainFFT.shape[1], 256)
    self.hidden2 = nn.Linear(256, 256)
    self.output = nn.Linear(256, 2)

  def forward(self, x):
    x = self.hidden(x)
    x = F.relu(x)
    x = self.hidden2(x)
    x = F.relu(x)
    x = self.output(x)
    return x
## Uncomment to test
#model = VanillaRNN(2, 5, trainFFT.shape[1])
model = Net1()
loss_func = nn.CrossEntropyLoss()
num_epochs = 1000
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate, weight_decay = 0.0001)
#traindataset = torch.utils.data.TensorDataset(torch.Tensor(np.expand_dims(trainChordDownSample40,1)), torch.Tensor(trainChordY))
#testdataset = torch.utils.data.TensorDataset(torch.Tensor(np.expand_dims(testChordDownSample40,1)), torch.Tensor(testChordY))

traindataset = torch.utils.data.TensorDataset(torch.Tensor(np.expand_dims(trainFFT,1)), torch.Tensor(trainChordY))
testdataset = torch.utils.data.TensorDataset(torch.Tensor(np.expand_dims(testFFT,1)), torch.Tensor(testChordY))

train_loader = torch.utils.data.DataLoader(traindataset, batch_size = 50, shuffle = True)
test_loader = torch.utils.data.DataLoader(testdataset, batch_size= 50, shuffle=False)
count = 0
loss_list = []
iteration_list = []
accuracy_list = []

for epoch in range(num_epochs):
  for datapoints, labels in train_loader:
    datapoints = datapoints.float()
    labels = labels.float()
    count += 1
    model.train()
    outputs = model(datapoints)
    loss = loss_func(np.squeeze(outputs), labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if (count < 10) or not(count % 300):
        with torch.no_grad():
            total = 0
            correct = 0
            for datapoints, labels in test_loader:
                model.eval()
                outputs = np.squeeze(model(datapoints))
                predictions = torch.max(outputs, 1)[1]
                truth = torch.max(labels, 1)[1]
                correct += (predictions == truth).sum().numpy()
                total += len(labels)
            accuracy = correct * 100 / total
            loss_list.append(loss.data)
            iteration_list.append(count)
            accuracy_list.append(accuracy)
            print(f'Epoch: {epoch+1:02d}, Iteration: {count:5d}, Loss: {loss.data:.4f}, Accuracy: {accuracy:.3f}%')

Epoch: 01, Iteration:     1, Loss: 7591.1855, Accuracy: 58.480%
Epoch: 01, Iteration:     2, Loss: 66486.9766, Accuracy: 58.480%
Epoch: 01, Iteration:     3, Loss: 42006.5664, Accuracy: 44.444%
Epoch: 01, Iteration:     4, Loss: 14299.5762, Accuracy: 42.105%
Epoch: 01, Iteration:     5, Loss: 20540.4375, Accuracy: 51.462%
Epoch: 01, Iteration:     6, Loss: 7038.3184, Accuracy: 58.480%
Epoch: 01, Iteration:     7, Loss: 10609.7266, Accuracy: 52.047%
Epoch: 01, Iteration:     8, Loss: 5488.6064, Accuracy: 35.673%
Epoch: 01, Iteration:     9, Loss: 4841.7236, Accuracy: 35.088%
Epoch: 22, Iteration:   300, Loss: 3.9676, Accuracy: 63.158%
Epoch: 43, Iteration:   600, Loss: 109.0068, Accuracy: 56.725%
Epoch: 65, Iteration:   900, Loss: 32.1180, Accuracy: 60.234%
Epoch: 86, Iteration:  1200, Loss: 5.1970, Accuracy: 60.234%
Epoch: 108, Iteration:  1500, Loss: 40.2084, Accuracy: 62.573%
Epoch: 129, Iteration:  1800, Loss: 14.4938, Accuracy: 63.158%
Epoch: 150, Iteration:  2100, Loss: 335.8056, 

In [2]:
print(A)

10


In [1]:
A = 10